# DETECTR BB QC 09-19-19 Analysis
This notebook is for formatting, analyzing and plotting the BB QC for BB made 09-19-19. 

## Objective
Ensure activity of old BB matches new BB

## Formatting to CSV file
Take txt file from plate reader and turn into tidy data format

In [2]:
#import needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

import holoviews as hv
import bokeh
import hvplot.pandas

In [3]:
#read in CSV file
DETECTR_raw = pd.read_csv("../raw_txt_files/DETECTR_BB_QC_09-19-19.txt", encoding='utf-16', sep="\t", delimiter="\t",index_col=None, skiprows= 3,header = None)

In [4]:
DETECTR_raw.tail(10)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
1164,NaN,NaN,NaN,15.578,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1165,NaN,NaN,NaN,1.477,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1166,NaN,NaN,NaN,16.073,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1167,NaN,NaN,NaN,1.401,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1168,~End,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1169,Original Filename: DETECTR_BB_QC_09-19-19; Dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1170,Workflow:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1171,"-Set Temperature: IsTemperatureControlOn=""True...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1172,"-Read: ExperimentNameToRead=""Expt1"" SectionNam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1173,"-Set Temperature: IsTemperatureControlOn=""True...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#drop unnecessary rows at the end of the csv
DETECTR_raw = DETECTR_raw.drop(list(range(len(DETECTR_raw)-6, len(DETECTR_raw))), axis=0)

In [6]:
#select only the first columns
DETECTR_raw = DETECTR_raw.iloc[:,0:4]

In [7]:
#how many plate reads did we have?
len(DETECTR_raw)/16

73.0

In [8]:
#set a variable to identify the number of plate reads
num_plate_reads = int(len(DETECTR_raw)/16)

In [9]:
#create a dictionary mapping the row number to a letter for 384 and 96
well_dict_384 = {1:"A",2:"B",3:"C",4:"D",5:"E",6:"F",7:"G",8:"H",9:"I",10:"J",11:"K",12:"L",13:"M",14:"N",15:"O",16:"P"}
well_dict_96 = {1:"A",2:"A",3:"B",4:"B",5:"C",6:"C",7:"D",8:"D",9:"E",10:"E",11:"F",12:"F",13:"G",14:"G",15:"H",16:"H"}

In [10]:
#initialize the cleaned data frame
DETECTR_cleaned = pd.DataFrame(columns = ['time','row_384','column_384', 'rfu_value'])

In [11]:
DETECTR_raw.head(10)

,0,1,2,3
0,00:00:00,36.9,NaN,2.932
1,NaN,NaN,NaN,1.393
2,NaN,NaN,NaN,2.891
3,NaN,NaN,NaN,1.597
4,NaN,NaN,NaN,2.813
5,NaN,NaN,NaN,1.731
6,NaN,NaN,NaN,3.258
7,NaN,NaN,NaN,1.745
8,NaN,NaN,NaN,5.146
9,NaN,NaN,NaN,1.891


In [12]:
#iterate over each plate to create a dataframe with the correct time, the 384 and 96 position and the RFU_value
for i in list(range(0,num_plate_reads)):
    time = DETECTR_raw.iloc[i*16,0]
    for j in list(range(3, 4,1)):
        for k in list(range(i*16, i*16+16)):
            rfu_value = DETECTR_raw.loc[k,j]
            row_384 = well_dict_384[k%16+1]
            column_384 = j-1
            #row_96 = well_dict_96[k%16+1]
            #column_96 = j // 2
            DETECTR_cleaned.loc[len(DETECTR_cleaned)] = [time, row_384, column_384,rfu_value]

In [13]:
DETECTR_cleaned.head(10)

,time,row_384,column_384,rfu_value
0,00:00:00,A,2,2.932
1,00:00:00,B,2,1.393
2,00:00:00,C,2,2.891
3,00:00:00,D,2,1.597
4,00:00:00,E,2,2.813
5,00:00:00,F,2,1.731
6,00:00:00,G,2,3.258
7,00:00:00,H,2,1.745
8,00:00:00,I,2,5.146
9,00:00:00,J,2,1.891


In [14]:
#how many rows in our data frame now? should be 384*number of plates
len(DETECTR_cleaned)

1168

In [15]:
DETECTR_cleaned=DETECTR_cleaned[DETECTR_cleaned["row_384"].isin(['A','C','E','G','I','K','M','O'])]

In [17]:
#write this file to a CSV
DETECTR_cleaned.to_csv("../tidy_data/DETECTR_BB_QC_09-19-19.csv")

In [18]:
DETECTR_cleaned[DETECTR_cleaned['row_384']=='A'].head(10)

,time,row_384,column_384,rfu_value
0,00:00:00,A,2,2.932
16,00:10:00,A,2,3.390
32,00:20:00,A,2,4.236
48,00:30:00,A,2,5.021
64,00:40:00,A,2,6.343
80,00:50:00,A,2,7.942
96,01:00:00,A,2,9.468
112,01:10:00,A,2,11.090
128,01:20:00,A,2,13.542
144,01:30:00,A,2,15.716


In [19]:
DETECTR_cleaned = pd.read_csv("../tidy_data/DETECTR_BB_QC_09-19-19.csv")

## Filter the data frame
Select only wells we are interested in

In [55]:
def filter_time(DETECTR_cleaned, time):
    DETECTR = DETECTR_cleaned.loc[DETECTR_cleaned["time"]==time]
    DETECTR = DETECTR.assign(well=(DETECTR['row_384'] + DETECTR['column_384'].map(str)))
    DETECTR_target = DETECTR.assign(target_DNA=['target_added','target_added','no_target','no_target']*2)
    DETECTR_target = DETECTR_target.assign(BB=['06-12-19']*4+['09-19-19']*4)
    DETECTR_target = DETECTR_target.assign(sample_set=(DETECTR_target['target_DNA'] + "_" + DETECTR_target['BB']))
    DETECTR_target = DETECTR_target.assign(replicate=(['a','b']*4))
    return DETECTR_target

In [73]:
def kinetics(DETECTR_cleaned, num_plate_reads):
    DETECTR = DETECTR_cleaned.assign(well=(DETECTR_cleaned['row_384'] + DETECTR_cleaned['column_384'].map(str)))
    DETECTR_target = DETECTR.assign(BB=num_plate_reads*(['06-12-19']*4+['09-19-19']*4))
    DETECTR_target = DETECTR_target.assign(target_DNA=num_plate_reads*(['target_added','target_added','no_target','no_target']*2))
    return DETECTR_target

In [57]:
#only select rows with the 2 hour time point 
DETECTR_2hr = filter_time(DETECTR_cleaned,"02:00:00")

In [58]:
DETECTR_2hr

,Unnamed: 0,time,row_384,column_384,rfu_value,well,target_DNA,BB,sample_set,replicate
96,192,02:00:00,A,2,25.483,A2,target_added,06-12-19,target_added_06-12-19,a
97,194,02:00:00,C,2,28.135,C2,target_added,06-12-19,target_added_06-12-19,b
98,196,02:00:00,E,2,2.696,E2,no_target,06-12-19,no_target_06-12-19,a
99,198,02:00:00,G,2,3.167,G2,no_target,06-12-19,no_target_06-12-19,b
100,200,02:00:00,I,2,123.291,I2,target_added,09-19-19,target_added_09-19-19,a
101,202,02:00:00,K,2,123.075,K2,target_added,09-19-19,target_added_09-19-19,b
102,204,02:00:00,M,2,2.840,M2,no_target,09-19-19,no_target_09-19-19,a
103,206,02:00:00,O,2,3.025,O2,no_target,09-19-19,no_target_09-19-19,b


In [60]:
DETECTR_2hr.hvplot(x='sample_set', y='rfu_value', by='replicate',rot = 90, kind = 'bar',hover_cols = ['well'] )

:Bars   [sample_set,replicate]   (rfu_value,well)

In [62]:
DETECTR_4hr = filter_time(DETECTR_cleaned,"04:00:00")

In [63]:
DETECTR_4hr.hvplot(x='sample_set', y='rfu_value', by='replicate',rot = 90, kind = 'bar',hover_cols = ['well'] )

:Bars   [sample_set,replicate]   (rfu_value,well)

In [64]:
DETECTR_30min = filter_time(DETECTR_cleaned,"00:30:00")

In [65]:
DETECTR_30min.hvplot(x='sample_set', y='rfu_value', by='replicate',rot = 90, kind = 'bar',hover_cols = ['well'] )

:Bars   [sample_set,replicate]   (rfu_value,well)

In [66]:
DETECTR_1hr = filter_time(DETECTR_cleaned,"01:00:00")

In [67]:
DETECTR_1hr.hvplot(x='sample_set', y='rfu_value', by='replicate',rot = 90, kind = 'bar',hover_cols = ['well'] )

:Bars   [sample_set,replicate]   (rfu_value,well)

In [75]:
DETECTR_kinetics = kinetics(DETECTR_cleaned,num_plate_reads)
DETECTR_kinetics['rfu_value'] = DETECTR_kinetics['rfu_value'].map(float)

DETECTR_kinetics.hvplot.scatter(x='time',y='rfu_value', by = 'BB', hover_cols = ['row_384'],rot = 90,height=500)

:NdOverlay   [BB]
   :Scatter   [time]   (rfu_value,row_384)

## Conclusions
+ Value of final saturation increases with increasing gRNA concentration
+ However, time of saturation does not increase
+ Synthetic RNA does not seem to be vastly different from IVT reagents

In [76]:
def max_value_and_time(DETECTR_kinetics):
    max_rfu_and_time_list = [];
    for well in DETECTR_kinetics['well'].unique():
        DETECTR_well = DETECTR_kinetics[DETECTR_kinetics['well']==well]
        max_rfu = DETECTR_well['rfu_value'].max()
        max_rfu_and_time_dict = { 'well' : well,
                                 'max_rfu' : max_rfu,
                                 'time' : DETECTR_well[DETECTR_well['rfu_value']==max_rfu]['time'].values[0],
                                'target_DNA' : DETECTR_well[DETECTR_well['rfu_value']==max_rfu]['target_DNA'].values[0],
                                'row_384' : DETECTR_well[DETECTR_well['rfu_value']==max_rfu]['row_384'].values[0]

                                }
        max_rfu_and_time_list.append(max_rfu_and_time_dict)

    return pd.DataFrame(max_rfu_and_time_list)


In [77]:
DETECTR_max = max_value_and_time(DETECTR_kinetics)

In [78]:
DETECTR_max=DETECTR_max.sort_values('time')

In [82]:
DETECTR_max.hvplot.bar(x = 'time',y ='max_rfu',by = 'well', hover_cols = ['target_DNA'],rot = 90)

:Bars   [time,well]   (max_rfu,target_DNA)

# Signal to Noise
Calculate the signal of each sample divided by the background

In [84]:
oldBBbackground = statistics.mean(list(DETECTR_2hr.loc[DETECTR_2hr['row_384'].isin(['E','G'])]['rfu_value'].map(float)))

In [85]:
newBBbackground = statistics.mean(list(DETECTR_2hr.loc[DETECTR_2hr['row_384'].isin(['M','O'])]['rfu_value'].map(float)))

In [90]:
DETECTR_2hr=DETECTR_2hr.assign(signal_to_noise=DETECTR_2hr['rfu_value']/newBBbackground)

In [93]:
DETECTR_2hr.hvplot(x='well',y='signal_to_noise',by='target_DNA', kind = 'bar', rot = 90)

:Bars   [well,target_DNA]   (signal_to_noise)

In [94]:
DETECTR_30min=DETECTR_30min.assign(signal_to_noise=DETECTR_30min['rfu_value']/newBBbackground)

In [95]:
DETECTR_30min.hvplot(x='well',y='signal_to_noise',by='target_DNA', kind = 'bar', rot = 90)

:Bars   [well,target_DNA]   (signal_to_noise)